<a href="https://colab.research.google.com/github/Tstrebe2/predicting-text-difficulty/blob/dave-updates/code/dave_feature_representations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install spacy --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
import pandas as pd
import numpy as np
import spacy
from collections import Counter
from spacy_readability import Readability

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [41]:
nlp = spacy.load("en_core_web_sm")
nlp.disable_pipes(['tok2vec', 'parser', 'attribute_ruler','ner'])
nlp.add_pipe('sentencizer')
# read = Readability()
# nlp.add_pipe('spacy_readability', last=True)
url = 'https://raw.githubusercontent.com/Tstrebe2/predicting-text-difficulty/main/assets/dale_chall.txt'
df = pd.read_csv(url,header=None, names=['word'])
dc_set = set(df['word'])



In [ ]:
!pip install spacy_readability

In [6]:
def D_ChallEvaluator(x,distinct=False):
  """ 
    x - str; the document to evaluate against Dale-Chall (D-C)
    distinct - boolean to determine whether each matching word counts once (True)
    or count each instance of a matching word (ie if the same dc word 
    appears twice or more)

    Returns  float representing the ratio of D-C words to total length of input 
    string
  """
  

  doc = nlp(x)
  
  # obtain the matched words between input str and the D-C set
  lemma_words = [token.lemma_ for token in doc]
  word_lst = set(lemma_words).intersection(dc_set)

  if distinct:
    # if we are interested in distinct words, just get the length of the set
    word_count = len(word_lst)

  else:  
    # if we look at instances, create a counter of matching D-C words and store
    # their frequencies.
    cntr = Counter([word for word in lemma_words if word in word_lst])
    
    # sum up all of the counts across the matching words
    word_count = np.sum([tup[1] for tup in cntr.items()])
  
  return word_count / len(x.split(' '))







In [9]:
train_url = 'https://raw.githubusercontent.com/Tstrebe2/predicting-text-difficulty/main/assets/WikiLarge_Train.csv'
train_df = pd.read_csv(train_url)

In [42]:
train_df['DC_Ratio'] = train_df['original_text'].apply(lambda x: D_ChallEvaluator(x,False))
train_df['Grade_level'] = train_df['original_text'].apply(lambda x: nlp(x)._.dale_chall)

In [31]:
nlp.pipe(train_df['original_text'].values )

0.1984